## MLOps 구축하기

In [1]:
import boto3
import json
from sagemaker import get_execution_role
from time import strftime
import calendar
import time

In [2]:
iam_client = boto3.client('iam')
role=get_execution_role()
base_role_name=role.split('/')[-1]

In [3]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']

### 2. MLOps에서 활용할 Policy 설정하기

해당 HOL에서 구현할 아키텍처에 필요한 managed policy를 아래와 같이 정의합니다. Role을 별도 생성하셔도 되지만 HOL의 편의성을 위해 SageMaker Notebook/Studio와 동일한 Role에 policy를 추가하여 계속 활용합니다.

In [4]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)

{'ResponseMetadata': {'RequestId': '4f0e5a9f-2179-4b34-b36c-3263a0a857a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4f0e5a9f-2179-4b34-b36c-3263a0a857a7',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 30 Dec 2021 13:20:54 GMT'},
  'RetryAttempts': 0}}

In [28]:
user_name = 'napkin2' ## 사용자 아이디
codecommit_cred = 'codecommit-cred'

In [29]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName='napkin2',
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

ServiceUserName : napkin2-at-687314952804 
ServicePassword : p3moUgUxxseAXd+n617TDzXFcRbrlcy3qB0GNhAnT8U=


In [30]:
sec_client = boto3.client('secretsmanager')

In [31]:
secret_string = json.dumps({
      "username": ServiceUserName,
      "password": ServicePassword
    })

In [33]:
sec_list = sec_client.list_secrets()['SecretList']

if len(sec_list) == 0:
    sec_response = sec_client.create_secret(
        Name=codecommit_cred,
        Description='This credential uses git_config for SageMaker in Lambda',
        SecretString=secret_string,
        Tags=[
            {
                'Key': 'Name',
                'Value': 'codecommit_credentials'
            },
        ]
    )
else:
    sec_response = sec_client.update_secret(
        SecretId=sec_list[0]['ARN'],
        SecretString=secret_string
    )

In [36]:
sec_arn = sec_response['ARN']

### 3. SageMaker Studio의 설정값에서 model_package_group_name 생성/가져오기
이 작업을 진행하기 이전에 SageMaker Studio에서 Project 생성이 필요합니다.

In [37]:
sm_client = boto3.client("sagemaker")

model_package_groups_list = sm_client.list_model_package_groups(
        SortBy='CreationTime',
        SortOrder='Ascending')

if len(model_package_groups_list['ModelPackageGroupSummaryList']) == 0:
    print("Creating a model package group for a new model")

    import time
    model_package_group_name = "yolov5-detect-" + str(round(time.time()))
    model_package_group_input_dict = {
     "ModelPackageGroupName" : model_package_group_name,
     "ModelPackageGroupDescription" : "Sample model package group"
    }

    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))
else:    
    model_package_group_name= model_package_groups_list['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
model_package_group_name

'yolov5-detect-1640243744'

In [38]:
# sm_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

In [39]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'yolov5-detect-1640243744',
   'ModelPackageVersion': 13,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:687314952804:model-package/yolov5-detect-1640243744/13',
   'ModelPackageDescription': 'Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)',
   'CreationTime': datetime.datetime(2021, 12, 30, 12, 1, 43, 648000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'yolov5-detect-1640243744',
   'ModelPackageVersion': 12,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:687314952804:model-package/yolov5-detect-1640243744/12',
   'ModelPackageDescription': 'Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)',
   'CreationTime': datetime.datetime(2021, 12, 30, 11, 45, 52, 283000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
 

### 4. Create Amazon EventBridge Rule

model registry에서 모델이 **Approved**되었을 때 이벤트 트리거를 만들기 위한 설정을 Amazon EventBridge Rule을 이용하여 설정합니다.

In [40]:
event_client = boto3.client('events')

In [41]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [42]:
rule_name = 'yolov5_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the yolov5 model',
)

### 5. Lambda function 생성

EventBridge 에서 Rule 만족하는 이벤트가 발생했을 때 실행되는 Lambda Function을 정의합니다. Lambda Function 은 테스트 데이터를 예측하는 Batch transform job을 수행하게 됩니다.


In [109]:
%store -r

In [110]:
lambda_client = boto3.client('lambda')

In [111]:
lambda_trust_policy=json.dumps({
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
})

In [112]:
role_name='lambda-assume-role_'+ strftime("%m%d-%H%M%s")
try:
    for role_list in iam_client.list_roles()['Roles']:
        pre_role_name = role_list['RoleName']
        if pre_role_name.split("_")[0] in ['lambda-assume-role']:
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
            )
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
            )
            iam_client.delete_role(RoleName=pre_role_name)
except:
    pass
finally:
    lambda_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=lambda_trust_policy
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
    )
    time.sleep(10)
role_name

'lambda-assume-role_1230-17291640885344'

In [113]:
# The name of our algorithm
algo_name = "yolov5-batch-transform"
lambda_path = "./sm_lambda_preprocessing"

In [114]:
# Get the region defined in the current configuration (default to us-west-2 if none defined)
my_session = boto3.session.Session()
region = my_session.region_name

repo_name=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{algo_name}:latest"

In [129]:
%%capture
!sh $lambda_path/build_and_push.sh $account_id $region $algo_name

In [123]:
lambda_name='yolov5-batch_transform'
try:
    lambda_client.delete_function(FunctionName=lambda_name)
except:
    pass
finally:
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_name,
        Role=lambda_role['Role']['Arn'],
        Code={
            'ImageUri': repo_name
        },
        PackageType='Image',
        Description='Create the latest version-based Informer model',
        Timeout=600,
        MemorySize=512,
        Environment={
          'Variables': {
              "sec_arn" : sec_arn
          }
      }
    )

In [130]:
# response = lambda_client.update_function_code(
#     FunctionName=lambda_name,
#     ImageUri=repo_name
# )

In [125]:
lambda_permission_response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='InvokeLambdaFunction',
    Action='lambda:InvokeFunction',
    Principal="events.amazonaws.com",
    SourceArn=event_rule['RuleArn'],
)

Amazon EventBridge에 위에서 생성한 Lambda function을 타켓으로 설정합니다.

In [126]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': lambda_response['FunctionArn']
        }
    ]
)

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': 'a05cfe84-687e-46f4-8190-ebb8cfdbffe3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a05cfe84-687e-46f4-8190-ebb8cfdbffe3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Thu, 30 Dec 2021 17:35:28 GMT'},
  'RetryAttempts': 0}}

## TEST - Register a Model Version

In [ ]:
model_package_group_name

In [ ]:
model_package_group_name='yolov5-detect-1640243744'

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": '687314952804.dkr.ecr.ap-northeast-2.amazonaws.com/yolov5:1.10.0-gpu-py38-inf',
         }
      ],
      "SupportedContentTypes": [ "application/x-image" ],
      "SupportedResponseMIMETypes": [ "application/x-image" ],
   }
 }
# Specify the model source
model_url = f"{artifacts_dir}model.tar.gz"

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]

create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

In [61]:
import os

In [80]:
w='./yolo_v5/model/test/model.tar.gz'

In [81]:
if w.endswith('.tar.gz'):
    print("AAA")

AAA


In [76]:
w

'./yolo_v5/model/test/model.tar.gz'

In [78]:
import tarfile
ap = tarfile.open(w)
ap.extractall("./yolo_v5/model/test/")
ap.close()

In [71]:
if os.path.isdir(w):
    w = os.path.join(w, "model.tar.gz")
    import tarfile
    ap = tarfile.open(w)
    ap.extractall("/opt/ml/processing/weights/")
    ap.close()
    weights = "/opt/ml/processing/weights/model/exp/weights/best.pt"
import glob
dest = glob.glob("./yolo_v5/model/*")
print(f" **************** dest : {dest}")

 **************** dest : ['./yolo_v5/model/models', './yolo_v5/model/test', './yolo_v5/model/exp']
